<a href="https://colab.research.google.com/github/varunmulay-droid/B-WAN/blob/main/WAN_21_I2V_14b_Working_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gradio


In [6]:
"""
Wan2.1 Image-to-Video Generation — Gradio GUI
4-tab pipeline: ENV SETUP | MODEL DOWNLOAD | INITIALIZATION | GENERATION

Key design decisions
─────────────────────
• Nothing runs on import — every step is user-triggered via buttons.
• torchvision is never imported at module level (Colab circular-import bug).
  run_initialization() purges the broken sys.modules stub and reimports
  torchvision in the correct C-extension order before ComfyUI loads.
• Gradio 6.x: theme passed to gr.Blocks(), CSS passed to demo.launch().
"""

import gradio as gr
import subprocess
import queue
import sys
import os
import gc
import random
import time

# ─────────────────────────────────────────────────────────────────────────────
#  Safe module-level pre-imports
#  torch alone is safe here; torchvision is intentionally skipped —
#  Colab leaves it partially initialised in sys.modules which causes
#  "has no attribute 'extension'" if touched before the purge in Tab 3.
# ─────────────────────────────────────────────────────────────────────────────
try:
    import torch
    import numpy as np
    from PIL import Image
    import imageio
except ImportError:
    torch = np = Image = imageio = None


def _safe_torch_import():
    """
    Re-entrant guard that returns a fully-initialised torch module.
    Falls back gracefully when packages are not yet installed.
    torchvision is intentionally left for ComfyUI's own import chain.
    """
    import importlib
    global torch, np, Image, imageio

    if torch is None or not hasattr(torch, "cuda"):
        torch = sys.modules.get("torch") or importlib.import_module("torch")
    if np is None:
        import numpy as _np;     np = _np          # noqa: E702
    if Image is None:
        from PIL import Image as _I; Image = _I    # noqa: E702
    if imageio is None:
        import imageio as _io;   imageio = _io     # noqa: E702
    return torch


# ─────────────────────────────────────────────────────────────────────────────
#  Shared pipeline state
# ─────────────────────────────────────────────────────────────────────────────
state: dict = {
    "env_ready":    False,
    "models_ready": False,
    "initialized":  False,
    "nodes":        {},
    "useQ6":        False,
}

log_queue: queue.Queue = queue.Queue()


# ─────────────────────────────────────────────────────────────────────────────
#  Helpers
# ─────────────────────────────────────────────────────────────────────────────
def stream_subprocess(cmd: str, log_fn) -> int:
    """Run a shell command and stream stdout/stderr line-by-line to log_fn."""
    proc = subprocess.Popen(
        cmd, shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1,
    )
    for line in iter(proc.stdout.readline, ""):
        log_fn(line.rstrip())
    proc.wait()
    return proc.returncode


# ═════════════════════════════════════════════════════════════════════════════
#  TAB 1 — ENVIRONMENT SETUP
# ═════════════════════════════════════════════════════════════════════════════
def run_env_setup(use_q6, progress=gr.Progress(track_tqdm=True)):
    state["useQ6"] = use_q6
    logs = []

    def log(msg):
        logs.append(msg)

    log("📦 Installing PyTorch 2.6.0 + torchvision …")
    yield "\n".join(logs), gr.update(interactive=False)

    rc = stream_subprocess("pip install torch==2.6.0 torchvision==0.21.0 -q", log)
    if rc != 0:
        log("❌ PyTorch installation failed.")
        yield "\n".join(logs), gr.update(interactive=True)
        return

    log("📦 Installing ComfyUI dependencies …")
    yield "\n".join(logs), gr.update(interactive=False)
    stream_subprocess(
        "pip install -q torchsde einops diffusers accelerate "
        "xformers==0.0.29.post2 av imageio imageio-ffmpeg",
        log,
    )

    log("📂 Cloning ComfyUI …")
    yield "\n".join(logs), gr.update(interactive=False)
    stream_subprocess(
        "git clone https://github.com/Isi-dev/ComfyUI /content/ComfyUI "
        "|| echo 'Already cloned'",
        log,
    )

    log("📂 Cloning ComfyUI_GGUF …")
    yield "\n".join(logs), gr.update(interactive=False)
    stream_subprocess(
        "git clone https://github.com/Isi-dev/ComfyUI_GGUF.git "
        "/content/ComfyUI/custom_nodes/ComfyUI_GGUF || echo 'Already cloned'",
        log,
    )
    stream_subprocess(
        "pip install -q -r /content/ComfyUI/custom_nodes/ComfyUI_GGUF/requirements.txt",
        log,
    )

    log("🔧 Installing system tools (aria2, ffmpeg) …")
    yield "\n".join(logs), gr.update(interactive=False)
    stream_subprocess("apt -y install -qq aria2 ffmpeg", log)

    state["env_ready"] = True
    log("✅ Environment setup complete!")
    yield "\n".join(logs), gr.update(
        interactive=True, value="✅ Setup Complete — proceed to Model Download"
    )


# ═════════════════════════════════════════════════════════════════════════════
#  TAB 2 — MODEL DOWNLOAD
# ═════════════════════════════════════════════════════════════════════════════
def run_model_download(use_q6, progress=gr.Progress()):
    logs = []

    def log(msg):
        logs.append(msg)

    if not state["env_ready"]:
        yield "⚠️  Please complete Environment Setup first.", gr.update(interactive=True)
        return

    model_q    = "Q6_K" if use_q6 else "Q4_0"
    model_file = f"wan2.1-i2v-14b-480p-{model_q}.gguf"
    model_url  = (
        f"https://huggingface.co/city96/Wan2.1-I2V-14B-480P-gguf"
        f"/resolve/main/{model_file}"
    )

    downloads = [
        (model_url,
         "/content/ComfyUI/models/unet", model_file,
         "🧠 UNet model"),
        ("https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/"
         "split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors",
         "/content/ComfyUI/models/text_encoders",
         "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
         "📝 Text encoder"),
        ("https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/"
         "split_files/vae/wan_2.1_vae.safetensors",
         "/content/ComfyUI/models/vae",
         "wan_2.1_vae.safetensors",
         "🎨 VAE"),
        ("https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/"
         "split_files/clip_vision/clip_vision_h.safetensors",
         "/content/ComfyUI/models/clip_vision",
         "clip_vision_h.safetensors",
         "👁️  CLIP Vision"),
    ]

    for url, dest_dir, fname, label in downloads:
        os.makedirs(dest_dir, exist_ok=True)
        dest = os.path.join(dest_dir, fname)
        if os.path.exists(dest):
            log(f"⏭️  {label} already exists — skipping.")
        else:
            log(f"⬇️  Downloading {label} …")
            yield "\n".join(logs), gr.update(interactive=False)
            rc = stream_subprocess(
                f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "
                f"{url} -d {dest_dir} -o {fname}",
                log,
            )
            if rc != 0:
                log(f"❌ Failed to download {label}")
                yield "\n".join(logs), gr.update(interactive=True)
                return
        yield "\n".join(logs), gr.update(interactive=False)

    state["models_ready"] = True
    log("✅ All models downloaded!")
    yield "\n".join(logs), gr.update(
        interactive=True, value="✅ Models Ready — proceed to Initialization"
    )


# ═════════════════════════════════════════════════════════════════════════════
#  TAB 3 — INITIALIZATION
# ═════════════════════════════════════════════════════════════════════════════
def run_initialization():
    logs = []

    def log(msg):
        logs.append(msg)

    if not state["models_ready"]:
        yield "⚠️  Please download models first.", gr.update(interactive=True)
        return

    log("🔄 Adding ComfyUI to Python path …")
    yield "\n".join(logs), gr.update(interactive=False)
    sys.path.insert(0, "/content/ComfyUI")

    # ── Torchvision circular-import fix ──────────────────────────────────────
    # Root cause: Colab's kernel partially imports torchvision at startup and
    # stores a broken half-constructed module in sys.modules.
    # importlib.reload() cannot fix it — a module whose __init__ never finished
    # cannot be re-entered safely.  The only reliable solution:
    #   1. Purge every torchvision.* key from sys.modules (remove broken stub)
    #   2. Re-import in C-extension-first order so _meta_registrations finds
    #      torchvision.extension fully bound when it calls _has_ops().
    log("🔧 Purging & reloading torchvision (circular-import fix) …")
    yield "\n".join(logs), gr.update(interactive=False)

    _tv_keys = [
        k for k in sys.modules
        if k == "torchvision" or k.startswith("torchvision.")
    ]
    for _k in _tv_keys:
        del sys.modules[_k]

    try:
        import torchvision._C          # step 1 — load compiled C extension
        import torchvision.extension   # step 2 — bind _has_ops() helper
        import torchvision             # step 3 — __init__ runs cleanly now
        log(f"✅ torchvision {torchvision.__version__} loaded cleanly.")
    except Exception as _tv_err:
        log(f"⚠️  torchvision reload warning (may still be OK): {_tv_err}")
    yield "\n".join(logs), gr.update(interactive=False)
    # ── end fix ──────────────────────────────────────────────────────────────

    try:
        log("🔄 Importing ComfyUI nodes …")
        yield "\n".join(logs), gr.update(interactive=False)

        from nodes import (                                           # noqa: E402
            CLIPLoader, CLIPTextEncode, VAEDecode,
            VAELoader, KSampler, UNETLoader,
            LoadImage, CLIPVisionLoader, CLIPVisionEncode,
        )
        from custom_nodes.ComfyUI_GGUF.nodes import UnetLoaderGGUF  # noqa: E402
        from comfy_extras.nodes_model_advanced import ModelSamplingSD3
        from comfy_extras.nodes_wan import WanImageToVideo

        state["nodes"] = {
            "unet_loader":          UnetLoaderGGUF(),
            "model_sampling":       ModelSamplingSD3(),
            "clip_loader":          CLIPLoader(),
            "clip_encode_positive": CLIPTextEncode(),
            "clip_encode_negative": CLIPTextEncode(),
            "vae_loader":           VAELoader(),
            "vae_decode":           VAEDecode(),
            "ksampler":             KSampler(),
            "clip_vision_loader":   CLIPVisionLoader(),
            "clip_vision_encode":   CLIPVisionEncode(),
            "load_image":           LoadImage(),
            "wan_i2v":              WanImageToVideo(),
        }

        log("✅ All nodes initialized successfully!")
        state["initialized"] = True
        yield "\n".join(logs), gr.update(
            interactive=True, value="✅ Initialized — proceed to Generation"
        )

    except Exception as e:
        log(f"❌ Initialization failed: {e}")
        yield "\n".join(logs), gr.update(interactive=True)


# ═════════════════════════════════════════════════════════════════════════════
#  TAB 4 — GENERATION
# ═════════════════════════════════════════════════════════════════════════════
def clear_memory():
    _torch = _safe_torch_import()
    gc.collect()
    if _torch and _torch.cuda.is_available():
        _torch.cuda.empty_cache()
        _torch.cuda.ipc_collect()
    gc.collect()


def generate_video_gui(
    image_path, positive_prompt, negative_prompt,
    width, height, seed, steps, cfg_scale,
    sampler_name, scheduler, frames, fps, output_format,
    progress=gr.Progress(),
):
    _torch = _safe_torch_import()

    logs = []

    def log(msg):
        logs.append(msg)

    if not state["initialized"]:
        yield "⚠️  Please initialize nodes first (Tab 3).", None
        return

    if image_path is None:
        yield "⚠️  Please upload an image.", None
        return

    nodes  = state["nodes"]
    use_q6 = state["useQ6"]

    if seed == 0:
        seed = random.randint(0, 2**32 - 1)

    log(f"🎲 Using seed: {seed}")
    yield "\n".join(logs), None

    output_dir = "/content/ComfyUI/output"
    os.makedirs(output_dir, exist_ok=True)

    try:
        with _torch.inference_mode():

            # ── [1/6] Text encoder ────────────────────────────────────────
            log("📝 [1/6] Loading text encoder …")
            yield "\n".join(logs), None
            progress(0.05, desc="Loading text encoder")

            clip     = nodes["clip_loader"].load_clip(
                "umt5_xxl_fp8_e4m3fn_scaled.safetensors", "wan", "default"
            )[0]
            positive = nodes["clip_encode_positive"].encode(clip, positive_prompt)[0]
            negative = nodes["clip_encode_negative"].encode(clip, negative_prompt)[0]
            del clip; _torch.cuda.empty_cache(); gc.collect()

            log("✅ Text encoder done.")
            yield "\n".join(logs), None

            # ── [2/6] Image + CLIP Vision ─────────────────────────────────
            log("🖼️  [2/6] Loading image & CLIP Vision …")
            yield "\n".join(logs), None
            progress(0.15, desc="Loading CLIP Vision")

            loaded_image       = nodes["load_image"].load_image(image_path)[0]
            clip_vision        = nodes["clip_vision_loader"].load_clip("clip_vision_h.safetensors")[0]
            clip_vision_output = nodes["clip_vision_encode"].encode(clip_vision, loaded_image, "none")[0]
            del clip_vision; _torch.cuda.empty_cache(); gc.collect()

            log("✅ Image & CLIP Vision done.")
            yield "\n".join(logs), None

            # ── [3/6] VAE encode ──────────────────────────────────────────
            log("🎨 [3/6] Loading VAE & encoding latents …")
            yield "\n".join(logs), None
            progress(0.25, desc="VAE encoding")

            vae = nodes["vae_loader"].load_vae("wan_2.1_vae.safetensors")[0]
            positive_out, negative_out, latent = nodes["wan_i2v"].encode(
                positive, negative, vae,
                int(width), int(height), int(frames), 1,
                loaded_image, clip_vision_output,
            )

            log("✅ VAE encoding done.")
            yield "\n".join(logs), None

            # ── [4/6] UNet ────────────────────────────────────────────────
            log(f"🧠 [4/6] Loading UNet ({'Q6_K' if use_q6 else 'Q4_0'}) …")
            yield "\n".join(logs), None
            progress(0.35, desc="Loading UNet")

            model_file = (
                "wan2.1-i2v-14b-480p-Q6_K.gguf"
                if use_q6 else
                "wan2.1-i2v-14b-480p-Q4_0.gguf"
            )
            model = nodes["unet_loader"].load_unet(model_file)[0]
            model = nodes["model_sampling"].patch(model, 8)[0]

            log("✅ UNet loaded.")
            yield "\n".join(logs), None

            # ── [5/6] Sampling ────────────────────────────────────────────
            log(f"⚙️  [5/6] Sampling ({steps} steps) …")
            yield "\n".join(logs), None
            progress(0.40, desc="Sampling …")

            def step_callback(step, total, _latent):
                pct = 0.40 + 0.45 * (step / max(total, 1))
                progress(pct, desc=f"Sampling step {step}/{total}")
                logs.append(f"   Step {step:>3}/{total}  ({int(pct * 100)}%)")

            sampled = nodes["ksampler"].sample(
                model=model,
                seed=seed,
                steps=int(steps),
                cfg=float(cfg_scale),
                sampler_name=sampler_name,
                scheduler=scheduler,
                positive=positive_out,
                negative=negative_out,
                latent_image=latent,
            )[0]

            del model; _torch.cuda.empty_cache(); gc.collect()
            log("✅ Sampling complete.")
            yield "\n".join(logs), None

            # ── [6/6] VAE decode ──────────────────────────────────────────
            log("🖥️  [6/6] Decoding latents …")
            yield "\n".join(logs), None
            progress(0.88, desc="VAE decoding")

            decoded = nodes["vae_decode"].decode(vae, sampled)[0]
            del vae; _torch.cuda.empty_cache(); gc.collect()

            # ── Save output ───────────────────────────────────────────────
            log(f"💾 Saving as {output_format.upper()} …")
            yield "\n".join(logs), None
            progress(0.95, desc="Saving")

            frames_np = [(img.cpu().numpy() * 255).astype(np.uint8) for img in decoded]
            ts = int(time.time())

            if int(frames) == 1:
                out_path = f"{output_dir}/output_{ts}.png"
                Image.fromarray(frames_np[0]).save(out_path)
            elif output_format.lower() == "mp4":
                out_path = f"{output_dir}/output_{ts}.mp4"
                with imageio.get_writer(out_path, fps=int(fps)) as writer:
                    for frame in frames_np:
                        writer.append_data(frame)
            else:  # webm
                out_path = f"{output_dir}/output_{ts}.webm"
                with imageio.get_writer(
                    out_path, format="FFMPEG", mode="I",
                    fps=int(fps), codec="vp9",
                    output_params=["-crf", "10"],
                ) as writer:
                    for frame in frames_np:
                        writer.append_data(frame)

            progress(1.0, desc="Done!")
            log(f"✅ Saved: {out_path}")
            yield "\n".join(logs), out_path

    except Exception as e:
        import traceback
        log(f"❌ Error: {e}")
        log(traceback.format_exc())
        yield "\n".join(logs), None
    finally:
        clear_memory()


# ═════════════════════════════════════════════════════════════════════════════
#  GRADIO THEMING ENGINE
#  Base: Ocean  |  Hues: sky / cyan / zinc  |  Fonts: Rajdhani + Share Tech Mono
#  radius_size / spacing_size go in the constructor (not .set())
# ═════════════════════════════════════════════════════════════════════════════
wan_theme = gr.themes.Ocean(
    primary_hue=gr.themes.colors.sky,
    secondary_hue=gr.themes.colors.cyan,
    neutral_hue=gr.themes.colors.zinc,
    radius_size=gr.themes.sizes.radius_md,
    spacing_size=gr.themes.sizes.spacing_md,
    font=[gr.themes.GoogleFont("Rajdhani"), "sans-serif"],
    font_mono=[gr.themes.GoogleFont("Share Tech Mono"), "monospace"],
).set(
    # ── Core palette ──────────────────────────────────────────────────────
    body_background_fill="#0a0c10",
    body_background_fill_dark="#0a0c10",
    body_text_color="#c8d6e5",
    body_text_color_dark="#c8d6e5",

    # ── Blocks / panels ───────────────────────────────────────────────────
    block_background_fill="#0d1b2a",
    block_background_fill_dark="#0d1b2a",
    block_border_color="#1e2d45",
    block_border_color_dark="#1e2d45",
    block_border_width="1px",
    block_radius="6px",
    block_shadow="0 4px 24px rgba(0,229,255,0.06)",
    block_shadow_dark="0 4px 24px rgba(0,229,255,0.06)",
    block_label_text_color="#7eb3d8",
    block_label_text_color_dark="#7eb3d8",
    block_label_text_size="0.75rem",
    block_title_text_color="#00e5ff",
    block_title_text_color_dark="#00e5ff",

    # ── Inputs ────────────────────────────────────────────────────────────
    input_background_fill="#06111e",
    input_background_fill_dark="#06111e",
    input_border_color="#1e3a5a",
    input_border_color_dark="#1e3a5a",
    input_border_color_focus="#00e5ff",
    input_border_color_focus_dark="#00e5ff",
    input_shadow_focus="0 0 8px rgba(0,229,255,0.25)",
    input_shadow_focus_dark="0 0 8px rgba(0,229,255,0.25)",
    input_radius="4px",

    # ── Buttons ───────────────────────────────────────────────────────────
    button_primary_background_fill="linear-gradient(135deg,#00517a,#007ea7)",
    button_primary_background_fill_dark="linear-gradient(135deg,#00517a,#007ea7)",
    button_primary_background_fill_hover="linear-gradient(135deg,#006f9f,#00b8d9)",
    button_primary_background_fill_hover_dark="linear-gradient(135deg,#006f9f,#00b8d9)",
    button_primary_border_color="#00b8d9",
    button_primary_border_color_dark="#00b8d9",
    button_primary_text_color="#ffffff",
    button_primary_text_color_dark="#ffffff",
    button_secondary_background_fill="#0d1b2a",
    button_secondary_background_fill_dark="#0d1b2a",
    button_secondary_border_color="#1e3a5a",
    button_secondary_border_color_dark="#1e3a5a",
    button_secondary_text_color="#4a8fa8",
    button_secondary_text_color_dark="#4a8fa8",
    button_large_radius="4px",
    button_small_radius="3px",

    # ── Slider ────────────────────────────────────────────────────────────
    slider_color="#00e5ff",
    slider_color_dark="#00e5ff",

    # ── Loader / progress ─────────────────────────────────────────────────
    loader_color="#00e5ff",
    loader_color_dark="#00e5ff",

    # ── Checkbox ──────────────────────────────────────────────────────────
    checkbox_background_color="#06111e",
    checkbox_background_color_dark="#06111e",
    checkbox_border_color="#1e3a5a",
    checkbox_border_color_dark="#1e3a5a",
    checkbox_label_background_fill="#0d1b2a",
    checkbox_label_background_fill_dark="#0d1b2a",

    # ── Table ─────────────────────────────────────────────────────────────
    table_even_background_fill="#0d1b2a",
    table_even_background_fill_dark="#0d1b2a",
    table_odd_background_fill="#06111e",
    table_odd_background_fill_dark="#06111e",
)


# ═════════════════════════════════════════════════════════════════════════════
#  CUSTOM CSS — Glassmorphism + neon layer (supplements theme, no logic)
# ═════════════════════════════════════════════════════════════════════════════
CUSTOM_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Share+Tech+Mono&family=Rajdhani:wght@400;600;700&display=swap');

/* ── CSS custom properties — single source of truth ── */
:root {
    --cyan:        #00e5ff;
    --cyan-dim:    #00b8d9;
    --cyan-glow:   rgba(0,229,255,0.18);
    --blue-deep:   #0a0c10;
    --blue-panel:  #0d1b2a;
    --blue-input:  #06111e;
    --blue-border: #1e3a5a;
    --blue-mid:    #1e2d45;
    --blue-muted:  #4a6fa5;
    --blue-label:  #7eb3d8;
    --text-body:   #c8d6e5;
    --text-input:  #a8c6e0;
    --green-term:  #39ff14;
    --green-border:#0d3320;
    --font-mono:   'Share Tech Mono', monospace;
    --font-sans:   'Rajdhani', sans-serif;
}

/* ── Global base ── */
body, .gradio-container, .main, footer {
    background: var(--blue-deep) !important;
}
body {
    color: var(--text-body) !important;
    font-family: var(--font-sans) !important;
}

/* deep-space ambient gradient overlay */
body::before {
    content: '';
    position: fixed; inset: 0; pointer-events: none; z-index: 0;
    background-image:
        radial-gradient(ellipse 80% 50% at 20% 10%, rgba(0,60,100,0.18) 0%, transparent 70%),
        radial-gradient(ellipse 60% 40% at 80% 90%, rgba(0,30,80,0.14) 0%, transparent 70%);
}

/* ── Glassmorphism panels ── */
.gr-block, .gr-box, .gr-panel,
.gradio-container .block, fieldset, .form {
    background: rgba(13,27,42,0.72) !important;
    backdrop-filter: blur(14px) saturate(160%) !important;
    -webkit-backdrop-filter: blur(14px) saturate(160%) !important;
    border: 1px solid rgba(0,229,255,0.10) !important;
    border-radius: 8px !important;
    box-shadow:
        0 2px 16px rgba(0,0,0,0.45),
        inset 0 1px 0 rgba(0,229,255,0.07) !important;
    transition: box-shadow 0.25s ease, border-color 0.25s ease !important;
}
.gr-block:hover, .gradio-container .block:hover {
    border-color: rgba(0,229,255,0.20) !important;
    box-shadow:
        0 4px 28px rgba(0,0,0,0.55),
        0 0 0 1px rgba(0,229,255,0.08),
        inset 0 1px 0 rgba(0,229,255,0.10) !important;
}

/* ── Tab bar — neon underline ── */
.tab-nav {
    background: rgba(6,17,30,0.85) !important;
    backdrop-filter: blur(10px) !important;
    border-bottom: 1px solid var(--blue-mid) !important;
    padding: 0 8px !important;
}
.tab-nav button {
    background: transparent !important;
    color: var(--blue-muted) !important;
    border: none !important;
    border-bottom: 2px solid transparent !important;
    border-radius: 0 !important;
    font-family: var(--font-mono) !important;
    font-size: 0.76rem !important;
    letter-spacing: 0.14em !important;
    text-transform: uppercase !important;
    padding: 12px 22px !important;
    transition: color 0.2s, border-color 0.2s, text-shadow 0.2s !important;
}
.tab-nav button.selected {
    color: var(--cyan) !important;
    border-bottom: 2px solid var(--cyan) !important;
    text-shadow: 0 0 10px var(--cyan-glow) !important;
    background: transparent !important;
}
.tab-nav button:hover:not(.selected) {
    color: var(--blue-label) !important;
    border-bottom-color: rgba(0,229,255,0.25) !important;
}

/* ── Inputs, textareas, select ── */
input:not([type=range]):not([type=checkbox]):not([type=radio]),
textarea, select, .gr-input, .gr-textarea {
    background: rgba(6,17,30,0.90) !important;
    backdrop-filter: blur(6px) !important;
    border: 1px solid var(--blue-border) !important;
    color: var(--text-input) !important;
    font-family: var(--font-mono) !important;
    border-radius: 4px !important;
    transition: border-color 0.2s, box-shadow 0.2s !important;
}
input:focus, textarea:focus {
    border-color: var(--cyan) !important;
    box-shadow: 0 0 0 2px rgba(0,229,255,0.15), 0 0 10px rgba(0,229,255,0.12) !important;
    outline: none !important;
}

/* ── Sliders — cyan glowing thumb ── */
input[type=range] { accent-color: var(--cyan) !important; }
input[type=range]::-webkit-slider-runnable-track {
    background: linear-gradient(90deg, #0d2a3a, var(--blue-border)) !important;
    height: 4px !important; border-radius: 2px !important;
}
input[type=range]::-webkit-slider-thumb {
    background: var(--cyan) !important;
    box-shadow: 0 0 8px var(--cyan-glow) !important;
    border: none !important;
    width: 14px !important; height: 14px !important;
    border-radius: 50% !important; margin-top: -5px !important;
    transition: box-shadow 0.15s !important;
}
input[type=range]::-webkit-slider-thumb:hover {
    box-shadow: 0 0 14px rgba(0,229,255,0.5) !important;
}
input[type=range]::-moz-range-track {
    background: linear-gradient(90deg, #0d2a3a, var(--blue-border)) !important;
    height: 4px !important; border-radius: 2px !important;
}
input[type=range]::-moz-range-thumb {
    background: var(--cyan) !important;
    box-shadow: 0 0 8px var(--cyan-glow) !important;
    border: none !important;
    width: 14px !important; height: 14px !important; border-radius: 50% !important;
}

/* ── Buttons — glassmorphism primary ── */
button.primary, .gr-button-primary, button[class*="primary"] {
    background: linear-gradient(135deg,
        rgba(0,81,122,0.90) 0%, rgba(0,126,167,0.90) 100%) !important;
    backdrop-filter: blur(8px) !important;
    color: #ffffff !important;
    border: 1px solid var(--cyan-dim) !important;
    font-family: var(--font-mono) !important;
    letter-spacing: 0.12em !important;
    text-transform: uppercase !important;
    border-radius: 4px !important;
    box-shadow: 0 0 12px rgba(0,184,217,0.30), inset 0 1px 0 rgba(255,255,255,0.08) !important;
    transition: box-shadow 0.2s, border-color 0.2s, transform 0.1s !important;
}
button.primary:hover, .gr-button-primary:hover {
    box-shadow: 0 0 22px rgba(0,229,255,0.45), inset 0 1px 0 rgba(255,255,255,0.12) !important;
    border-color: var(--cyan) !important;
    transform: translateY(-1px) !important;
}
button.primary:active { transform: translateY(0) !important; }

button.secondary, .gr-button-secondary, button[class*="secondary"] {
    background: rgba(13,27,42,0.80) !important;
    backdrop-filter: blur(6px) !important;
    color: var(--blue-muted) !important;
    border: 1px solid var(--blue-border) !important;
    font-family: var(--font-mono) !important;
    border-radius: 4px !important;
}

/* ── Terminal log boxes ── */
.log-box textarea {
    background: rgba(2,12,20,0.97) !important;
    backdrop-filter: blur(4px) !important;
    color: var(--green-term) !important;
    font-family: var(--font-mono) !important;
    font-size: 0.71rem !important;
    line-height: 1.6 !important;
    border: 1px solid var(--green-border) !important;
    border-radius: 4px !important;
    box-shadow: inset 0 0 30px rgba(57,255,20,0.04) !important;
    text-shadow: 0 0 4px rgba(57,255,20,0.35) !important;
}

/* ── Labels ── */
label span, .gr-label, .label-wrap span {
    color: var(--blue-label) !important;
    font-size: 0.77rem !important;
    letter-spacing: 0.07em !important;
    text-transform: uppercase !important;
}
.section-title {
    color: var(--cyan) !important;
    font-family: var(--font-mono) !important;
    font-size: 0.83rem !important;
    letter-spacing: 0.22em !important;
    text-transform: uppercase !important;
    border-bottom: 1px solid var(--blue-mid);
    padding-bottom: 6px; margin-bottom: 14px;
    text-shadow: 0 0 10px rgba(0,229,255,0.30);
}

/* ── Video / image output ── */
video, .gr-video, .image-container img {
    border: 1px solid rgba(0,184,217,0.30) !important;
    border-radius: 6px !important;
    box-shadow: 0 0 24px rgba(0,184,217,0.12),
                inset 0 0 0 1px rgba(0,229,255,0.05) !important;
}

/* ── Progress bar ── */
.progress-bar, .progress-bar > div,
[class*="progress"] > [class*="fill"], .eta-bar {
    background: linear-gradient(90deg, #005a8a, var(--cyan)) !important;
}
.loader, [class*="loader"] { color: var(--cyan) !important; }

/* ── Checkbox & radio ── */
input[type=checkbox]:checked, input[type=radio]:checked {
    accent-color: var(--cyan) !important;
}
.gr-checkbox-label, .gr-radio-row label {
    color: var(--text-body) !important;
    font-family: var(--font-sans) !important;
}

/* ── Dropdown ── */
.gr-dropdown, .svelte-select, [class*="dropdown"] {
    background: rgba(6,17,30,0.92) !important;
    border: 1px solid var(--blue-border) !important;
    color: var(--text-input) !important;
    font-family: var(--font-mono) !important;
}
.svelte-select .listContainer, [class*="dropdown-content"] {
    background: rgba(10,16,26,0.98) !important;
    border: 1px solid var(--blue-mid) !important;
    backdrop-filter: blur(10px) !important;
}

/* ── Scrollbar ── */
::-webkit-scrollbar { width: 5px; height: 5px; }
::-webkit-scrollbar-track { background: var(--blue-input); }
::-webkit-scrollbar-thumb { background: var(--blue-border); border-radius: 3px; }
::-webkit-scrollbar-thumb:hover { background: var(--cyan-dim); }

/* ── Status badge ── */
.status-badge {
    font-family: var(--font-mono) !important;
    font-size: 0.73rem !important;
    padding: 4px 12px !important;
    border-radius: 3px !important;
    letter-spacing: 0.06em !important;
}
"""


# ═════════════════════════════════════════════════════════════════════════════
#  GRADIO LAYOUT
# ═════════════════════════════════════════════════════════════════════════════
with gr.Blocks(title="Wan2.1 I2V — Control Panel", theme=wan_theme) as demo:

    gr.HTML("""
    <div style="text-align:center;padding:28px 0 18px;
                font-family:'Share Tech Mono',monospace;
                background:linear-gradient(180deg,#00060e 0%,transparent 100%);">
        <div style="font-size:2rem;letter-spacing:0.35em;color:#00e5ff;
                    text-shadow:0 0 20px #00e5ffaa;font-weight:700;">
            WAN 2.1&nbsp;&nbsp;I2V
        </div>
        <div style="font-size:0.72rem;letter-spacing:0.5em;color:#4a6fa5;margin-top:4px;">
            IMAGE &middot; TO &middot; VIDEO &middot; GENERATION &middot; PIPELINE
        </div>
    </div>
    """)

    with gr.Tabs():

        # ── TAB 1 — ENV SETUP ─────────────────────────────────────────────
        with gr.Tab("① ENV SETUP"):
            gr.HTML('<div class="section-title">Environment Configuration</div>')
            with gr.Row():
                with gr.Column(scale=1):
                    q6_env = gr.Checkbox(
                        label="Use Q6_K model (higher quality, needs more VRAM)",
                        value=False,
                    )
                    gr.HTML("""
                    <div style="font-family:'Share Tech Mono',monospace;font-size:0.72rem;
                                color:#4a6fa5;margin-top:8px;line-height:1.8;">
                        This step will:<br>
                        &nbsp;▸ Install PyTorch 2.6.0 + torchvision<br>
                        &nbsp;▸ Install ComfyUI dependencies<br>
                        &nbsp;▸ Clone ComfyUI + ComfyUI_GGUF repos<br>
                        &nbsp;▸ Install aria2 &amp; ffmpeg
                    </div>
                    """)
                    env_btn    = gr.Button("▶  RUN ENVIRONMENT SETUP", variant="primary")
                    env_status = gr.Button("⬜ Not Started", variant="secondary", interactive=False)
                with gr.Column(scale=2):
                    env_log = gr.Textbox(
                        label="Setup Log", lines=20, max_lines=30,
                        interactive=False, elem_classes=["log-box"],
                    )
            env_btn.click(run_env_setup, inputs=[q6_env], outputs=[env_log, env_status])

        # ── TAB 2 — MODEL DOWNLOAD ────────────────────────────────────────
        with gr.Tab("② MODEL DOWNLOAD"):
            gr.HTML('<div class="section-title">Model Weights Download</div>')
            with gr.Row():
                with gr.Column(scale=1):
                    q6_dl = gr.Checkbox(
                        label="Use Q6_K model (must match ENV SETUP setting)",
                        value=False,
                    )
                    gr.HTML("""
                    <div style="font-family:'Share Tech Mono',monospace;font-size:0.72rem;
                                color:#4a6fa5;margin-top:8px;line-height:1.8;">
                        Downloads:<br>
                        &nbsp;▸ UNet (Q4_0 ~8 GB / Q6_K ~12 GB)<br>
                        &nbsp;▸ Text encoder UMT5-XXL FP8<br>
                        &nbsp;▸ VAE  wan_2.1<br>
                        &nbsp;▸ CLIP Vision H
                    </div>
                    """)
                    dl_btn    = gr.Button("⬇  DOWNLOAD ALL MODELS", variant="primary")
                    dl_status = gr.Button("⬜ Not Started", variant="secondary", interactive=False)
                with gr.Column(scale=2):
                    dl_log = gr.Textbox(
                        label="Download Log", lines=20, max_lines=30,
                        interactive=False, elem_classes=["log-box"],
                    )
            dl_btn.click(run_model_download, inputs=[q6_dl], outputs=[dl_log, dl_status])

        # ── TAB 3 — INITIALIZATION ────────────────────────────────────────
        with gr.Tab("③ INITIALIZATION"):
            gr.HTML('<div class="section-title">Node Initialization</div>')
            with gr.Row():
                with gr.Column(scale=1):
                    gr.HTML("""
                    <div style="font-family:'Share Tech Mono',monospace;font-size:0.72rem;
                                color:#4a6fa5;margin-top:8px;line-height:1.8;">
                        Loads into memory:<br>
                        &nbsp;▸ UnetLoaderGGUF<br>
                        &nbsp;▸ CLIPLoader / CLIPTextEncode<br>
                        &nbsp;▸ VAELoader / VAEDecode<br>
                        &nbsp;▸ CLIPVisionLoader / Encode<br>
                        &nbsp;▸ KSampler / WanImageToVideo<br>
                        &nbsp;▸ ModelSamplingSD3
                    </div>
                    """)
                    init_btn    = gr.Button("⚡  INITIALIZE NODES", variant="primary")
                    init_status = gr.Button("⬜ Not Started", variant="secondary", interactive=False)
                with gr.Column(scale=2):
                    init_log = gr.Textbox(
                        label="Initialization Log", lines=20, max_lines=30,
                        interactive=False, elem_classes=["log-box"],
                    )
            init_btn.click(run_initialization, inputs=[], outputs=[init_log, init_status])

        # ── TAB 4 — GENERATION ────────────────────────────────────────────
        with gr.Tab("④ GENERATION"):
            gr.HTML('<div class="section-title">Image → Video Generation</div>')
            with gr.Row():
                with gr.Column(scale=1):
                    input_image = gr.Image(label="Input Image", type="filepath", height=240)

                    positive_prompt = gr.Textbox(
                        label="Positive Prompt", lines=3,
                        value="A cute anime girl with massive fennec ears and a big fluffy tail "
                              "wearing a maid outfit turning around.",
                    )
                    negative_prompt = gr.Textbox(
                        label="Negative Prompt", lines=3,
                        value="Bright colors, overexposed, static, blurred details, subtitles, "
                              "style, artwork, painting, frame, still, overall grayish, worst "
                              "quality, low quality, JPEG artifacts, ugly, mutilated, extra "
                              "fingers, poorly drawn hands, poorly drawn face, deformed, "
                              "disfigured, malformed limbs, fused fingers, motionless frame, "
                              "cluttered background, three legs, crowded background, walking backwards.",
                    )

                    with gr.Row():
                        width  = gr.Slider(256, 1280, step=64, value=832, label="Width")
                        height = gr.Slider(256, 720,  step=64, value=480, label="Height")
                    with gr.Row():
                        frames = gr.Slider(1, 120, step=1, value=33, label="Frames")
                        fps    = gr.Slider(1,  60, step=1, value=16, label="FPS")
                    with gr.Row():
                        steps     = gr.Slider(1, 100,  step=1,   value=20,  label="Steps")
                        cfg_scale = gr.Slider(1.0, 20.0, step=0.5, value=1.0, label="CFG Scale")

                    seed = gr.Number(label="Seed  (0 = random)", value=0, precision=0)

                    with gr.Row():
                        sampler_name = gr.Dropdown(
                            ["uni_pc", "euler", "dpmpp_2m", "ddim", "lms"],
                            value="uni_pc", label="Sampler",
                        )
                        scheduler = gr.Dropdown(
                            ["simple", "normal", "karras", "exponential"],
                            value="simple", label="Scheduler",
                        )

                    output_format = gr.Radio(["mp4", "webm"], value="mp4", label="Output Format")
                    gen_btn = gr.Button("🎬  GENERATE VIDEO", variant="primary", size="lg")

                with gr.Column(scale=1):
                    gen_log = gr.Textbox(
                        label="Generation Log  [ live ]",
                        lines=18, max_lines=28,
                        interactive=False, elem_classes=["log-box"],
                    )
                    output_video = gr.Video(label="Output Video", height=320)

            gen_btn.click(
                generate_video_gui,
                inputs=[
                    input_image, positive_prompt, negative_prompt,
                    width, height, seed, steps, cfg_scale,
                    sampler_name, scheduler, frames, fps, output_format,
                ],
                outputs=[gen_log, output_video],
            )

    gr.HTML("""
    <div style="text-align:center;padding:16px 0 8px;
                font-family:'Share Tech Mono',monospace;font-size:0.65rem;
                color:#1e3a5a;letter-spacing:0.2em;">
        WAN 2.1 &middot; COMFYUI BACKEND &middot; GGUF QUANTIZED
    </div>
    """)


if __name__ == "__main__":
    demo.launch(
        share=True,
        css=CUSTOM_CSS,  # glassmorphism + neon layer — Gradio 6.x: CSS in launch()
    )

/tmp/ipython-input-14682/2788933664.py:825: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Wan2.1 I2V — Control Panel", theme=wan_theme) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad1df00742d31eb395.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
